<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# მოდელის გამოყენება და პროგნოზირება (Inference)

ამ Notebook-ის მიზანია, გამოვიყენოთ ჩვენი საუკეთესო მოდელი (**PatchTST**), მოვამზადოთ სატესტო მონაცემები, გავაკეთოთ პროგნოზი და დავაგენერიროთ `submission.csv` ფაილი Kaggle-ზე ასატვირთად.

## 1. გარემოს მომზადება

ეს ნაწილი ამზადებს გარემოს: ვამაგრებთ Google Drive-ს, ვაყენებთ საჭირო ბიბლიოთეკებს და ვიწერთ Kaggle-ის მონაცემებს.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q wandb kaggle pandas numpy scikit-learn neuralforecast joblib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 285.8/285.8 kB 27.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
! mkdir -p ~/.kaggle
! cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

! kaggle competitions download -c walmart-recruiting-store-sales-forecasting
! unzip -o walmart-recruiting-store-sales-forecasting.zip > /dev/null
! rm walmart-recruiting-store-sales-forecasting.zip
! unzip -oq '*.zip'

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 671MB/s]

4 archives were successfully processed.


In [4]:
import pandas as pd
import numpy as np
import joblib
import wandb
from sklearn.base import BaseEstimator, RegressorMixin
from neuralforecast import NeuralForecast
from neuralforecast.models import PatchTST

## 2. მოდელის და დამხმარე კლასების ჩატვირთვა

სანამ პროგნოზს გავაკეთებთ, აუცილებელია ჩავტვირთოთ შენახული მოდელის ობიექტი Weights & Biases-დან და განვსაზღვროთ ის დამხმარე კლასები, რომლებიც ტრენინგის დროს გამოვიყენეთ. `joblib`-ს სჭირდება ამ კლასების დეფინიცია, რომ სწორად შეძლოს მოდელის ობიექტის აღდგენა.

In [5]:
# ეს კლასი ზუსტად უნდა იმეორებდეს იმ კლასს, რომელიც PatchTST მოდელის ტრენინგისას გამოვიყენეთ.
# მისი არსებობა აუცილებელია, რომ joblib-მა შეძლოს შენახული მოდელის კორექტულად აღდგენა.
class NeuralForecastModels(BaseEstimator, RegressorMixin):
    def __init__(self, models, model_names, freq='W-FRI', group_cols=['Store', 'Dept'], one_model=False, date_col='Date'):
        self.models = models
        self.model_names = model_names
        self.freq = freq
        self.group_cols = group_cols
        self.date_col = date_col
        self.nf = None
        self.fitted = False
        self.one_model=one_model

    def _prepare_df(self, X, y=None):
        df = X.copy()
        df['ds'] = pd.to_datetime(df[self.date_col])
        df['unique_id'] = df[self.group_cols].astype(str).agg('-'.join, axis=1)
        if y is not None:
            df['y'] = y.values if isinstance(y, pd.Series) else y
            return df[['unique_id', 'ds', 'y']]
        else:
            return df[['unique_id', 'ds']]

    def fit(self, X, y):
        df = self._prepare_df(X, y)
        self.nf = NeuralForecast(models=self.models, freq=self.freq)
        self.nf.fit(df)
        self.fitted = True
        return self

    def predict(self, X_test):
        if not self.fitted:
            raise ValueError("Model is not fitted. Call fit() first.")
        forecast = self.nf.predict()
        return forecast

In [6]:
# ავტორიზაცია wandb-ზე
!wandb login

# ჩაწერეთ თქვენი პროექტის და არტეფაქტის სახელი
PROJECT_NAME = "Walmart Recruiting - Store Sales Forecasting" # თქვენი პროექტის სახელი
ARTIFACT_NAME = "lchik22-free-uni/Walmart Recruiting - Store Sales Forecasting/patch_tst:v0" # ფორმატი: [entity]/[project]/[artifact_name]:[version]

run = wandb.init(project=PROJECT_NAME, job_type="inference")
artifact = run.use_artifact(ARTIFACT_NAME, type='model')
artifact_dir = artifact.download()

model_path = f"{artifact_dir}/patch_tst_01.pkl"
loaded_model = joblib.load(model_path)

print(f"მოდელი '{ARTIFACT_NAME}' წარმატებით ჩაიტვირთა {artifact_dir}-დან.")

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: lchik22 (lchik22-free-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Currently logged in as: lchik22 (lchik22-free-uni) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb:   1 of 1 files downloaded.  


მოდელი 'lchik22-free-uni/Walmart Recruiting - Store Sales Forecasting/patch_tst:v0' წარმატებით ჩაიტვირთა /content/artifacts/patch_tst:v0-დან.


## 3. პროგნოზირება და Submission ფაილის გენერაცია

ჩატვირთული მოდელის გამოყენებით ვაკეთებთ პროგნოზს და ვქმნით `submission.csv` ფაილს.

In [7]:
# სატესტო მონაცემების მომზადება არ არის საჭირო, რადგან nf.predict() იყენებს შიდა ისტორიას
print("პროგნოზირების პროცესი იწყება...")
predictions_df = loaded_model.predict(None) # None არგუმენტი უთითებს, რომ გამოიყენოს დასწავლილი მონაცემების ბოლო ნაწილი
print("პროგნოზი წარმატებით დასრულდა.")

# Submission ფაილის ფორმატირება
submission_template = pd.read_csv('/content/sampleSubmission.csv')
test_df = pd.read_csv('/content/test.csv')

# შევქმნათ Id სვეტი სატესტო მონაცემებისთვის
test_df['Id'] = test_df['Store'].astype(str) + '_' + test_df['Dept'].astype(str) + '_' + test_df['Date']

# პროგნოზირებული მონაცემების მომზადება
predictions_df_formatted = predictions_df.copy()
predictions_df_formatted[['Store', 'Dept']] = predictions_df_formatted['unique_id'].str.split('-', expand=True)
predictions_df_formatted['Store'] = predictions_df_formatted['Store'].astype(int)
predictions_df_formatted['Dept'] = predictions_df_formatted['Dept'].astype(int)
predictions_df_formatted['Date'] = predictions_df_formatted['ds'].dt.strftime('%Y-%m-%d')

predictions_df_formatted['Id'] = predictions_df_formatted['Store'].astype(str) + '_' + \
                                 predictions_df_formatted['Dept'].astype(str) + '_' + \
                                 predictions_df_formatted['Date']

predictions_df_formatted = predictions_df_formatted.rename(columns={'PatchTST': 'Weekly_Sales'})
predictions_df_formatted['Weekly_Sales'] = predictions_df_formatted['Weekly_Sales'].clip(lower=0)

# საბოლოო submission ფაილის შექმნა
submission_df = pd.merge(test_df[['Id']], predictions_df_formatted[['Id', 'Weekly_Sales']], on='Id', how='left')
submission_df.fillna(0, inplace=True)

submission_df.to_csv('submission.csv', index=False)

print("\nsubmission.csv ფაილი წარმატებით შეიქმნა.")
print("შეგიძლიათ ჩამოტვირთოთ ფაილი და ატვირთოთ Kaggle-ზე.")
submission_df.head()

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


პროგნოზირების პროცესი იწყება...


INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


პროგნოზი წარმატებით დასრულდა.

submission.csv ფაილი წარმატებით შეიქმნა.
შეგიძლიათ ჩამოტვირთოთ ფაილი და ატვირთოთ Kaggle-ზე.


,Id,Weekly_Sales
0,1_1_2012-11-02,32364.679688
1,1_1_2012-11-09,21673.224609
2,1_1_2012-11-16,20853.441406
3,1_1_2012-11-23,21174.871094
4,1_1_2012-11-30,23190.933594


## 4. Kaggle-ზე ატვირთვა (Command Line)

შეგიძლიათ გამოიყენოთ Kaggle-ის API `submission.csv` ფაილის პირდაპირ Colab-იდან ასატვირთად.

In [8]:
!kaggle competitions submit -c walmart-recruiting-store-sales-forecasting -f submission.csv -m "Final PatchTST Model Submission"

100% 2.88M/2.88M [00:00<00:00, 4.07MB/s]
Successfully submitted to Walmart Recruiting - Store Sales Forecasting